<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/diffusers/Controlnetdiffusers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Github](https://github.com/lllyasviel/ControlNet-v1-1-nightly)

[usage](https://huggingface.co/lllyasviel/sd-controlnet-depth)

[Annotator](https://github.com/patrickvonplaten/controlnet_aux)


Installation

In [ ]:
!pip install -q diffusers accelerate transformers controlnet_aux

Get Demo Data

In [ ]:
!wget https://images.thdstatic.com/productImages/091e153a-245e-4f21-ab67-ac5dbc3437b8/svn/dark-grey-lifestyle-solutions-sofas-couches-ccwenks3m26dgra-64_600.jpg -O /content/sofa.jpg

Load model

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch
from diffusers import UniPCMultistepScheduler



# controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_canny", torch_dtype=torch.float16)
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_mlsd", torch_dtype=torch.float16)
# controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11f1p_sd15_depth", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "SG161222/Realistic_Vision_V5.1_noVAE", controlnet=controlnet, torch_dtype=torch.float16
).to("cuda")

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

Load estimator

In [ ]:
from controlnet_aux import HEDdetector, MidasDetector, MLSDdetector, OpenposeDetector, PidiNetDetector, NormalBaeDetector, LineartDetector, LineartAnimeDetector, CannyDetector, ContentShuffleDetector, ZoeDetector, MediapipeFaceDetector, SamDetector, LeresDetector, DWposeDetector
from transformers import pipeline
# load checkpoints
# hed = HEDdetector.from_pretrained("lllyasviel/Annotators")
midas = pipeline('depth-estimation')
mlsd = MLSDdetector.from_pretrained("lllyasviel/Annotators")
# open_pose = OpenposeDetector.from_pretrained("lllyasviel/Annotators")
# pidi = PidiNetDetector.from_pretrained("lllyasviel/Annotators")
# normal_bae = NormalBaeDetector.from_pretrained("lllyasviel/Annotators")
# lineart = LineartDetector.from_pretrained("lllyasviel/Annotators")
# lineart_anime = LineartAnimeDetector.from_pretrained("lllyasviel/Annotators")
# zoe = ZoeDetector.from_pretrained("lllyasviel/Annotators")
# sam = SamDetector.from_pretrained("ybelkada/segment-anything", subfolder="checkpoints")
# mobile_sam = SamDetector.from_pretrained("dhkim2810/MobileSAM", model_type="vit_t", filename="mobile_sam.pt")
# leres = LeresDetector.from_pretrained("lllyasviel/Annotators")

Main function

In [ ]:
import cv2
from PIL import Image
import numpy as np

image_path = "/content/sofa.jpg"
image = Image.open(image_path)
bh, bw = image.size

image = image.resize((512, 512))



# for canny
# low_threshold = 100
# high_threshold = 200

# image = np.array(image)
# image = cv2.Canny(image, low_threshold, high_threshold)
# image = image[:, :, None]
# image = np.concatenate([image, image, image], axis=2)
# pre_image = Image.fromarray(image)

# for mlsd
pre_image = mlsd(image)
pre_image = pre_image.resize((512, 512))

# for depth
# pre_image = midas(image)["depth"]
# pre_image = np.array(pre_image)
# pre_image = pre_image[:, :, None]
# pre_image = np.concatenate([pre_image, pre_image, pre_image], axis=2)
# pre_image = Image.fromarray(pre_image)



prompt = "milky sofa"
generator = torch.Generator(device="cpu").manual_seed(-1)

new_image = pipe(
    prompt,
    pre_image,
    negative_prompt="monochrome, lowres, bad anatomy, worst quality, low quality",
    num_inference_steps=20,
    generator=generator,
).images[0]

display(new_image)

In [ ]:
new_image.resize((bh, bw)).save("/content/sofa_white.jpg")